# Función princiapal cgurin_responde
### Sección 1: Descripción General de la Función

La función cgurin_responde está diseñada para manejar consultas sobre seguridad minera subterránea. A través de una serie de pasos, procesa la consulta del usuario, identifica el nivel relevante de la mina, y luego busca información pertinente desde un índice vectorial en Pinecone. Dependiendo de la naturaleza de la consulta, devuelve planos de la mina o respuestas detalladas sobre emergencias y condiciones en la mina.

Parámetros:
- query: la consulta del usuario.
- emergencia (opcional): información sobre una posible emergencia.
- last_update (opcional): el estado más reciente de los sensores de la mina.
- Valor de Retorno:
- Una cadena de texto que contiene la respuesta generada, que puede ser un archivo de imagen o una respuesta textual.

In [1]:
# # Análisis de la Función cgurin_responde

# Este notebook está diseñado para desglosar y explicar la función `cgurin_responde`, que se utiliza para responder a consultas relacionadas con la seguridad en la minería subterránea, haciendo uso de Pinecone, OpenAI y datos de sensores de la mina.

# ## Importación de librerías necesarias
# Primero, se importan las librerías que se utilizarán en la función. Algunas de ellas, como `os` y `dotenv`, son útiles para cargar variables de entorno y gestionar configuraciones de API.

import os
from dotenv import load_dotenv
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec
import re
import base64
import tempfile
from PIL import Image
from io import BytesIO

# ## Carga de variables de entorno

# La función empieza cargando las variables de entorno desde un archivo `.env`, lo cual es fundamental para obtener las claves de acceso a los servicios de Pinecone y OpenAI. Si alguna clave falta, se lanza un error.

# Cargar archivo .env con las claves de API
load_dotenv()

# Obtener la clave API de Pinecone
api_key_pinecone = os.getenv("api_key_pinecone")
if not api_key_pinecone:
    raise ValueError("La variable de entorno 'api_key_pinecone' no está definida en el archivo .env.")

# Inicializar Pinecone con la clave API
pc = Pinecone(api_key=api_key_pinecone)

# Obtener la clave API de OpenAI
api_key_openai = os.getenv("api_key_openai")
if not api_key_openai:
    raise ValueError("La variable de entorno 'api_key_openai' no está definida en el archivo .env.")

# Configura la clave API de OpenAI
client = OpenAI(api_key=api_key_openai)


### Sección 2: Procesamiento de la Consulta
Se convierte la consulta a minúsculas para normalizar la entrada y facilitar la búsqueda.
Se definen los niveles de la mina en una lista (niveles).
A través de una expresión regular, se extrae el número de nivel que aparece en la consulta, si existe. Si no se encuentra un número, se asigna el valor "general".

In [ ]:
# Convertir la consulta a minúsculas para normalizar la entrada
query_lower = query.lower()

# Definir los niveles disponibles en la mina
niveles = ["1950", "1850", "1810", "1750", "1900", "general"]

# Extraer el número de nivel de la consulta utilizando una expresión regular
number_in_query = re.search(r'\d+', query)
nivel = number_in_query.group() if number_in_query else "general"


### Sección 3: Búsqueda de Información sobre Planos
Explicación:
- Si la consulta está relacionada con planos de la mina, se realiza una consulta a OpenAI para obtener un embedding de la consulta. Este embedding es luego utilizado para realizar una búsqueda en Pinecone, que devuelve los resultados más relevantes.
- La búsqueda en Pinecone filtra por imágenes codificadas en base64.

In [ ]:
# Caso en que la consulta busca información sobre un plano de algún nivel
if ('plano' in query_lower or 'lageplan' in query_lower or 'plane' in query_lower) and any(nivel in query_lower for nivel in niveles):
    # Llamada a OpenAI para obtener el embedding de la consulta
    response_imagen = client.embeddings.create(
        input=query,
        model="text-embedding-3-small"
    )

    # Realizar la consulta al índice vectorial de Pinecone para buscar resultados de planos
    results_imagen = index.query(
        namespace="ns1",
        vector=response_imagen.data[0].embedding,
        top_k=3,  # Número de resultados a devolver
        include_values=True,
        include_metadata=True,
        filter={"imagen_b64": {"$ne": None, "$ne": '', "$ne": " "}}  # Filtrar por aquellos que tengan imagen en base64
    )


### Sección 4: Decodificación de Imágenes y Respuesta
- Los resultados de la búsqueda son filtrados para asegurarse de que coincidan con el nivel específico y que contengan la palabra "plano".
- Si se encuentran resultados válidos, la imagen en base64 es decodificada y convertida a un archivo temporal de imagen, que luego es retornado.

In [ ]:
# Filtrar los resultados para asegurarse de que coincidan con el nivel y contengan "plano"
filtered_results = []
if nivel:  # Verificar si 'nivel' tiene un valor válido antes de realizar el filtrado
    for match in results_imagen['matches']:
        if nivel in match['id'] and 'plano' in match['id']:
            filtered_results.append(match)

# Verificar si hay resultados filtrados
if filtered_results:
    # Extraer la imagen en base64 del primer resultado filtrado
    imagen_b64 = filtered_results[0]["metadata"].get("imagen_b64", "")
    
    if imagen_b64:  # Solo proceder si se encuentra una imagen en base64 válida
        # Decodificar la imagen base64
        imagen_data = base64.b64decode(imagen_b64)
        
        # Convertir a formato de imagen con PIL
        image = Image.open(BytesIO(imagen_data))
        
        # Crear un archivo temporal y guardar la imagen
        with tempfile.NamedTemporaryFile(delete=False, suffix=".png") as temp_file:
            image.save(temp_file, format="PNG")  # Guardar la imagen en el archivo temporal
            temp_file_path = temp_file.name  # Obtener la ruta del archivo temporal
        
        # Retornar la ruta del archivo temporal
        return temp_file_path


### Sección 5: Respuesta para Consultas Generales o Emergencias
- Si la consulta no está relacionada con un plano, se busca información textual utilizando Pinecone y OpenAI.
- Se filtran los resultados para asegurarse de que no contengan imágenes.
- Se prepara un mensaje para OpenAI con información adicional si la consulta es sobre una emergencia, y se obtiene una respuesta generada por el modelo GPT.

In [ ]:
else:
    # Si la consulta no es sobre un plano, se procede a consultar la información textual
    response = client.embeddings.create(
        input=query,
        model="text-embedding-3-small"
    )

    # Realizar la consulta al índice vectorial de Pinecone para buscar información relevante
    results = index.query(
        namespace="ns1",
        vector=response.data[0].embedding,
        top_k=5,  # Número de resultados a devolver
        include_values=False,
        include_metadata=True
    )

    # Filtrar los resultados para asegurar que el 'nivel' esté en el 'id' y que no contengan imágenes
    filtered_results = []
    for match in results['matches']:
        imagen_b64 = match['metadata'].get("imagen_b64", "")
        if imagen_b64 in [None, "", " "]:  # Solo considerar resultados sin imagen
            filtered_results.append(match)

    # Si hay resultados filtrados, preparar el mensaje para el modelo GPT
    if filtered_results:
        messages = [
            {
                "role": "system",  # Mensaje del sistema con información básica sobre el contexto
                "content": "Eres un experto en seguridad de minería subterránea..."
            },
            {
                "role": "user",  # Mensaje del usuario con la consulta
                "content": f"El usuario te dará un mensaje como este: \"{query}\"."
            }
        ]

        # Si el diccionario 'emergencia' se proporciona, agregarlo al mensaje
        if emergencia:
            messages.append({
                "role": "user",  # Mensaje adicional con detalles de la emergencia
                "content": f"Información adicional sobre la emergencia: {emergencia}"
            })

        # Llamada a OpenAI para obtener una respuesta basada en el modelo GPT-4
        response = client.chat.completions.create(
            model="gpt-4o",  # Asegúrate de que estés utilizando el modelo adecuado
            messages=messages,
            temperature=0.9,
            max_tokens=2048
        )

        # Almacenar y retornar la respuesta generada por GPT
        respuesta = response.choices[0].message.content
        return respuesta
